In [10]:
import ee

# Initialize Earth Engine
ee.Authenticate(force=True)

Enter verification code:  4/1Ab32j900mmn33MOI7TY7q5LqZOB-WOltLL6CPlE4tNl0XHXR0oelrUO1Ga8



Successfully saved authorization token.


In [13]:
import geopandas as gpd
import ee
import json

# Simple initialization - if this fails, we'll deal with it differently
ee.Initialize(project='ee-zc-povertymapping')

# Load your local shapefile
zamboanga_city = gpd.read_file('../assets/shapefile/zc04AdminBoundaries_gcs.shp')

print(zamboanga_city.head())
print(f"CRS: {zamboanga_city.crs}")
print(f"Features: {len(zamboanga_city)}")

# Convert shapefile to GeoJSON manually (avoid geemap)
zamboanga_geojson = json.loads(zamboanga_city.to_json())

# Convert GeoJSON to Earth Engine geometry
zamboanga_geom = ee.Geometry(zamboanga_geojson['features'][0]['geometry'])

# Cloud masking function
def mask_s2_clouds(img):
    qa = img.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return img.updateMask(mask)

# Build Sentinel-2 composite
sentinel2024 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2024-01-01', '2024-12-31')
    .filterBounds(zamboanga_geom)
    .map(mask_s2_clouds)
    .map(lambda img: img.select(['B2','B3','B4','B8','B11','B12'])
                        .divide(10000)
                        .copyProperties(img, img.propertyNames()))
    .median()
    .clip(zamboanga_geom))

# Export to Google Drive
task = ee.batch.Export.image.toDrive(
    image=sentinel2024,
    description='Sentinel2_2024_ZamboangaCity',
    folder='povertymap_exports',
    fileNamePrefix='sentinel2_2024_zamboanga_city',
    region=zamboanga_geom,
    scale=10,
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)
task.start()
print('Export task started!')

   psgc_code    adm4_pcode       adm4_en  shape_len  shape_area  shape_sqkm  \
0  907332001  PH0907332001  Arena Blanco   0.087205    0.000077    0.941164   
1  907332002  PH0907332002         Ayala   0.106437    0.000332    4.051912   
2  907332004  PH0907332004     Baliwasan   0.064013    0.000104    1.273695   
3  907332005  PH0907332005        Baluno   0.285519    0.002761   33.738149   
4  907332010  PH0907332010        Boalan   0.150027    0.000660    8.061650   

                                            geometry  
0  MULTIPOLYGON (((122.16 6.9147, 122.16 6.9147, ...  
1  POLYGON ((121.94 6.9863, 121.95 6.9849, 121.95...  
2  POLYGON ((122.06 6.9214, 122.06 6.9213, 122.06...  
3  POLYGON ((122.03 7.0478, 122.03 7.0465, 122.03...  
4  POLYGON ((122.11 6.9777, 122.11 6.9772, 122.11...  
CRS: EPSG:4326
Features: 101
Export task started!


In [17]:
import os
import subprocess

# Load and prepare shapefile
zamboanga_city = gpd.read_file('../assets/shapefile/zc04AdminBoundaries_gcs.shp')

# Ensure WGS84
if zamboanga_city.crs.to_epsg() != 4326:
    zamboanga_city = zamboanga_city.to_crs('EPSG:4326')

# Save as GeoJSON with absolute path
temp_geojson = os.path.abspath('temp_zamboanga.geojson')
zamboanga_city.to_file(temp_geojson, driver='GeoJSON')
print(f"GeoJSON saved to: {temp_geojson}")

# Upload using earthengine command line
asset_id = 'projects/your-project-id/assets/zamboanga_city'  # Change this!

cmd = f'earthengine upload table --asset_id={asset_id} "{temp_geojson}"'
result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

print(result.stdout)
if result.returncode == 0:
    print(f"✓ Upload started successfully!")
    print(f"Asset ID: {asset_id}")
    print("Check status: https://code.earthengine.google.com/ -> Tasks tab")
else:
    print(f"✗ Error: {result.stderr}")

# Don't delete yet - wait for upload to complete
print(f"\nKeep the file at {temp_geojson} until upload completes")
print("Then manually delete it or run: os.remove(temp_geojson)")

GeoJSON saved to: C:\Users\Admin\povertymapping\dataprep\temp_zamboanga.geojson
Invalid Cloud Storage URL: C:\Users\Admin\povertymapping\dataprep\temp_zamboanga.geojson

✗ Error: 

Keep the file at C:\Users\Admin\povertymapping\dataprep\temp_zamboanga.geojson until upload completes
Then manually delete it or run: os.remove(temp_geojson)


In [19]:
# Load and prepare shapefile
zamboanga_city = gpd.read_file('../assets/shapefile/zc04AdminBoundaries_gcs.shp')

print(f"Loaded {len(zamboanga_city)} features")
print(f"CRS: {zamboanga_city.crs}")

# Ensure WGS84
if zamboanga_city.crs.to_epsg() != 4326:
    print("Reprojecting to EPSG:4326...")
    zamboanga_city = zamboanga_city.to_crs('EPSG:4326')

# Convert to GeoJSON format
geojson_dict = json.loads(zamboanga_city.to_json())

# Create Earth Engine FeatureCollection from GeoJSON
features = []
for feature in geojson_dict['features']:
    ee_feature = ee.Feature(
        ee.Geometry(feature['geometry']),
        feature['properties']
    )
    features.append(ee_feature)

fc = ee.FeatureCollection(features)

# Define asset ID - change this to your path!
asset_id = 'projects/your-project-id/assets/zamboanga_city'

# Export as asset
task = ee.batch.Export.table.toAsset(
    collection=fc,
    description='Upload_Zamboanga_City',
    assetId=asset_id
)

task.start()

print(f"✓ Upload task started!")
print(f"Asset ID: {asset_id}")
print(f"Check task status at: https://code.earthengine.google.com/ -> Tasks tab")
print(f"Once completed, load it with: ee.FeatureCollection('{asset_id}')")

Loaded 101 features
CRS: EPSG:4326


EEException: Google Earth Engine API has not been enabled in project: your-project-id.

In [21]:
import geopandas as gpd
import ee
import json

# Initialize with your actual project ID
ee.Initialize(project='ee-zc-povertymapping')

# Load and prepare shapefile
zamboanga_city = gpd.read_file('../assets/shapefile/zc04AdminBoundaries_gcs.shp')

print(f"Loaded {len(zamboanga_city)} features")
print(f"CRS: {zamboanga_city.crs}")

# Ensure WGS84
if zamboanga_city.crs.to_epsg() != 4326:
    print("Reprojecting to EPSG:4326...")
    zamboanga_city = zamboanga_city.to_crs('EPSG:4326')

# Convert to GeoJSON format
geojson_dict = json.loads(zamboanga_city.to_json())

# Create Earth Engine FeatureCollection from GeoJSON
features = []
for feature in geojson_dict['features']:
    ee_feature = ee.Feature(
        ee.Geometry(feature['geometry']),
        feature['properties']
    )
    features.append(ee_feature)

fc = ee.FeatureCollection(features)

# Define asset ID with your project
asset_id = 'projects/ee-zc-povertymapping/assets/zamboanga_city'

# Export as asset
task = ee.batch.Export.table.toAsset(
    collection=fc,
    description='Upload_Zamboanga_City',
    assetId=asset_id
)

task.start()

print(f"✓ Upload task started!")
print(f"Asset ID: {asset_id}")
print(f"Check task status at: https://code.earthengine.google.com/ -> Tasks tab")
print(f"\nOnce completed, load it with:")
print(f"zamboanga_geom = ee.FeatureCollection('{asset_id}').geometry()")

Loaded 101 features
CRS: EPSG:4326
✓ Upload task started!
Asset ID: projects/ee-zc-povertymapping/assets/zamboanga_city
Check task status at: https://code.earthengine.google.com/ -> Tasks tab

Once completed, load it with:
zamboanga_geom = ee.FeatureCollection('projects/ee-zc-povertymapping/assets/zamboanga_city').geometry()


In [22]:
import geopandas as gpd
import ee
import json

# Initialize with your project ID
ee.Initialize(project='ee-zc-povertymapping')

# Load the grid cells GeoJSON
grid_cells = gpd.read_file('../assets/grid_cells.geojson')

print(f"Loaded {len(grid_cells)} grid cells")
print(f"CRS: {grid_cells.crs}")
print(grid_cells.head())

# Ensure WGS84
if grid_cells.crs.to_epsg() != 4326:
    print("Reprojecting to EPSG:4326...")
    grid_cells = grid_cells.to_crs('EPSG:4326')

# Convert to GeoJSON format
geojson_dict = json.loads(grid_cells.to_json())

# Create Earth Engine FeatureCollection from GeoJSON
features = []
for feature in geojson_dict['features']:
    ee_feature = ee.Feature(
        ee.Geometry(feature['geometry']),
        feature['properties']
    )
    features.append(ee_feature)

fc = ee.FeatureCollection(features)

# Define asset ID
asset_id = 'projects/ee-zc-povertymapping/assets/grid_cells'

# Export as asset
task = ee.batch.Export.table.toAsset(
    collection=fc,
    description='Upload_Grid_Cells',
    assetId=asset_id
)

task.start()

print(f"✓ Upload task started!")
print(f"Asset ID: {asset_id}")
print(f"Check task status at: https://code.earthengine.google.com/ -> Tasks tab")
print(f"\nOnce completed, load it with:")
print(f"grid_cells = ee.FeatureCollection('{asset_id}')")

Loaded 1724 grid cells
CRS: EPSG:4326
  grid_id  x_idx  y_idx                                           geometry
0    17_0     17      0  POLYGON ((122.05 6.8581, 122.06 6.8581, 122.06...
1    18_0     18      0  POLYGON ((122.06 6.8581, 122.07 6.8581, 122.07...
2    19_0     19      0  POLYGON ((122.07 6.8581, 122.08 6.8581, 122.08...
3    16_1     16      1  POLYGON ((122.04 6.8671, 122.05 6.8671, 122.05...
4    17_1     17      1  POLYGON ((122.05 6.8671, 122.06 6.8671, 122.06...
✓ Upload task started!
Asset ID: projects/ee-zc-povertymapping/assets/grid_cells
Check task status at: https://code.earthengine.google.com/ -> Tasks tab

Once completed, load it with:
grid_cells = ee.FeatureCollection('projects/ee-zc-povertymapping/assets/grid_cells')


In [23]:
task = download_sentinel2_gee(bounds, '2024-01-01', '2024-12-31', output_path)
task.start()

NameError: name 'download_sentinel2_gee' is not defined